In [54]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1.- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2.- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [55]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea


In [56]:
df_attack=attack[(attack["country"]=="usa") | (attack["country"]=="australia")|\
          (attack["country"]=="south africa") | (attack["country"]=="papua new guinea")|\
          (attack["country"]=="new zealand")]

In [57]:
df_attack.rename(columns={'country':'id_paises'}, inplace=True)

/tmp/ipykernel_14736/4288119105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.rename(columns={'country':'id_paises'}, inplace=True)


In [58]:
df_attacks2=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])

/tmp/ipykernel_14736/3667869154.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_attacks2=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])


- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [59]:
df['rh_profile']=df['rh_profile'].apply(ast.literal_eval)
rh = df['rh_profile'].apply(pd.Series)

for i in range(len(rh.columns)):
    rh_col0=rh[i].apply(pd.Series)
    nombre="rh"+ rh_col0["layer"][115]
    valores= list(rh_col0["rh"])
    df.insert(i, nombre, valores)
    


In [60]:
df=df.drop(['rh_profile'], axis=1)


In [61]:
#Wind direction

df['wind_profile']=df['wind_profile'].apply(ast.literal_eval)
wind = df['wind_profile'].apply(pd.Series)

for i in range(len(wind.columns)):
    wind_col0=wind[i].apply(pd.Series)
    nombre="wind_"+ wind_col0["layer"][1]
    valores= list(wind_col0["direction"])
    df.insert(i, nombre, valores)


In [62]:
df=df.drop(['wind_profile'], axis=1)

In [63]:
df2=df.groupby('paises').agg(['mean'])

/tmp/ipykernel_14736/1401032147.py:1: FutureWarning: ['prec_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df2=df.groupby('paises').agg(['mean'])


In [64]:
df2=df2.reset_index()

In [65]:
df2.rename(columns={'paises':'id_paises'}, inplace=True)

In [66]:
def minuscula(col):
    try:
        col=col.lower()
        return col
    except:
        pass



In [67]:
df2["id_paises"]=df2["id_paises"].apply(minuscula)

In [68]:
merge_final=pd.merge(left=df2, right=df_attacks2, how='left', left_on='id_paises', right_on='id_paises')

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [69]:
merge_final.to_csv('datos/merge_clima_ataques.csv')

### Creamos las funciones de nuestro código para automatizar todo.

In [70]:
def limpiar_attack (df):
    df_attack=df[(df["country"]=="usa") | (df["country"]=="australia")|\
          (df["country"]=="south africa") | (df["country"]=="papua new guinea")|\
          (df["country"]=="new zealand")]
    df_attack.rename(columns={'country':'id_paises'}, inplace=True)
    df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
    df_final=df_final.reset_index()
    return df_final

In [72]:
def limpieza_clima(df):
    #Limpiamos la columna de rh_profile primero a series y luego a columnas en el bbucle for
    df['rh_profile']=df['rh_profile'].apply(ast.literal_eval)
    rh = df['rh_profile'].apply(pd.Series)

    for i in range(len(rh.columns)):
        rh_col0=rh[i].apply(pd.Series)
        nombre="rh"+ rh_col0["layer"][115]
        valores= list(rh_col0["rh"])
        df.insert(i, nombre, valores)
    #Eliminamos la columna que ya no necesitamos
    df=df.drop(['rh_profile'], axis=1)
    
    #Hacemos lo mismo para la columnas wind_profile
    df['wind_profile']=df['wind_profile'].apply(ast.literal_eval)
    wind = df['wind_profile'].apply(pd.Series)

    for i in range(len(wind.columns)):
        wind_col0=wind[i].apply(pd.Series)
        nombre="wind_"+ wind_col0["layer"][1]
        valores= list(wind_col0["direction"])
        df.insert(i, nombre, valores)
        
    df=df.drop(['wind_profile'], axis=1)
    
    df2=df.groupby('paises').agg(['mean'])
    df2=df2.reset_index()
    df2.rename(columns={'paises':'id_paises'}, inplace=True)
    def minuscula(col):
        try:
            col=col.lower()
            return col
        except:
            pass
    df2["id_paises"]=df2["id_paises"].apply(minuscula)

    return df2

In [ ]:
def merge(df_left,df_rigth): #Esta función está persanda para hacer un left. Para que se tenga en cuenta
    merge_final=pd.merge(left=df_left, right=df_rigth, how='left', left_on='id_paises', right_on='id_paises')
    return merge_final